In [ ]:
from fmu.sumo.explorer import Explorer

%matplotlib inline

### Initialize Explorer

In [ ]:
sumo = Explorer()

### Find cases

In [ ]:
# Get Drogon cases
myassetname = "Drogon"  # Must be a valid asset on Sumo
cases = sumo.cases.filter(asset=myassetname)

# Get available status filters
print("Statuses:", cases.statuses)

# Filter on status
cases = cases.filter(status="keep")

# Get available user filters
print("Users:", cases.users)

# Filer on user
cases = cases.filter(user="dbs")

# Iterate over results
print("\nResults:", len(cases))

for case in cases:
    print("ID:", case.uuid)
    print("Name:", case.name)
    print("\n")

# Get case by name (name is not guaranteed to be unique)
mycasename = cases[0].name  # for sake of example
case = sumo.cases.filter(name=mycasename)[0]

# Get case by id
mycaseuuid = cases[0].uuid  # for sake of example
case = sumo.cases.filter(uuid=mycaseuuid)[0]

# Select case
case = cases[0]

print("\nSelected case:")
print("ID:", case.uuid)
print("Name:", case.name)
print("Status:", case.status)
print("Asset:", case.asset)
print("Field:", case.field)
print("User:", case.user)
print("Surfaces:", len(case.surfaces))
print("Polygons:", len(case.polygons))
print("Tables:", len(case.polygons))

### Find data


In [ ]:
# Get case surfaces
surfs = case.surfaces.filter(realization=True)

# Get available ensembles
print("Ensembles:", surfs.ensembles)

# Filter on ensemble
surfs = surfs.filter(ensemble="iter-0")

# Get available contents
print("Contents:", surfs.contents)

# Filter on content
surfs = surfs.filter(content="depth")

# Get available names
print("Names:", surfs.names)

# Filter on name
surfs = surfs.filter(name="Valysar Fm.")

# Filter on format
surfs = surfs.filter(dataformat="irap_binary")

# Get available tagnames
print("Tagnames:", surfs.tagnames)

# Filter on tagnames
surfs = surfs.filter(tagname="FACIES_Fraction_Channel")

# Iterate over results
print("\nResults:", len(surfs))

for surf in surfs:
    print("\n")
    print("ID:", surf.uuid)
    print("Format:", surf.dataformat)
    print("Realization:", surf.realization)

# Select one surface instance
surf = surfs[0]

print("\nSelected surface:")
print("Uuid:", surf.uuid)
print("Name:", surf.name)
print("Content:", surf.content)
print("Tagname:", surf.tagname)
print("Format:", surf.dataformat)
print("Ensemble:", surf.ensemble)
print("Realization:", surf.realization)
print("vertical domain:", surf.vertical_domain)

# xtgeo.RegularSurface
reg_surf = surf.to_regular_surface()
reg_surf.quickplot()

### Seismic - OpenVDS

In [ ]:
# Case containing seismic cubes
seismic_case = sumo.filter(has={"term": {"class.keyword": "cube"}})[0]

cubes = seismic_case.cubes

print("Ensembles\t", cubes.ensembles)

cubes.filter(ensemble="iter-0")

print("Names:\t\t", cubes.names)

cubes = cubes.filter(name="seismic")

print("Tagnames:\t", cubes.tagnames)

cubes = cubes.filter(tagname="amplitude_depth")

print("Num cubes:\t", len(cubes))

# Select cube instance
cube = cubes[0]

print("\nSelected cube:")
print("ID:", cube.uuid)
print("Name:", cube.name)
print("Tagname:", cube.tagname)
print("Ensemble:", cube.ensemble)
print("Realization:", cube.realization)
print("is_observation:", cube["data"]["is_observation"])
print("is_prediction:", cube["data"]["is_prediction"])

print("\nOpenVDS data:")
openvds_handle = cube.openvds_handle

layout = openvds.getLayout(openvds_handle)
channel_count = layout.getChannelCount()
print("Channel count: ", channel_count)
print("Channel names: ")
for i in range(channel_count):
    print("   ", layout.getChannelName(i))

### Aggregate

In [ ]:
# Perform aggregation on SurfaceCollection

regsurf = surfs.aggregate(
    operation="min"
)  # operations: max, mean, std, p10, p90, p50
regsurf.to_regular_surface().quickplot()

### Time filtration
The `TimeFilter` class can be used to construct a time filter which can be passed to the `SurfaceCollection.filter` method.

In [ ]:
from fmu.sumo.explorer import TimeFilter, TimeType

# get surfaces with timestamps
time = TimeFilter(time_type=TimeType.TIMESTAMP)
surfs = case.surfaces.filter(time=time)
print("Timestamp:", len(surfs))

# get surfaces with time intervals
time = TimeFilter(time_type=TimeType.INTERVAL)
surfs = case.surfaces.filter(time=time)
print("Interval:", len(surfs))


# get surfaces with time data (timestamp or interval)
time = TimeFilter(time_type=TimeType.ALL)
surfs = case.surfaces.filter(time=time)
print("Time data:", len(surfs))


# get surfaces without time data
time = TimeFilter(time_type=TimeType.NONE)
surfs = case.surfaces.filter(time=time)
print("No time data:", len(surfs))


# get avaiable timestamps
timestamps = case.surfaces.timestamps
print("\nTimestamps:", timestamps)

# get available intervals
intervals = case.surfaces.intervals
print("Intervals:", intervals)


# get surfaces with timestamp in range
time = TimeFilter(
    time_type=TimeType.TIMESTAMP, start="2018-01-01", end="2022-01-01"
)
surfs = case.surfaces.filter(time=time)

# get surfaces with time intervals in range
time = TimeFilter(
    time_type=TimeType.INTERVAL, start="2018-01-01", end="2022-01-01"
)
surfs = case.surfaces.filter(time=time)

# get surfaces where intervals overlap with range
time = TimeFilter(
    time_type=TimeType.INTERVAL,
    start="2018-01-01",
    end="2022-01-01",
    overlap=True,
)
surfs = case.surfaces.filter(time=time)

# get surfaces with exact timestamp matching (t0 == start)
time = TimeFilter(time_type=TimeType.TIMESTAMP, start="2018-01-01", exact=True)
surfs = case.surfaces.filter(time=time)

# get surfaces with exact interval matching (t0 == start AND t1 == end)
time = TimeFilter(
    time_type=TimeType.INTERVAL,
    start="2018-01-01",
    end="2022-01-01",
    exact=True,
)
surfs = case.surfaces.filter(time=time)